In [29]:
import os 

os.chdir("app/")

FileNotFoundError: [Errno 2] No such file or directory: 'app/'

In [2]:
import pandas as pd
import numpy as np

import json
import glob
import hydra
from hydra import initialize, compose

import torch

from ptls.preprocessing import PandasDataPreprocessor
from ptls.data_load.iterable_processing import SeqLenFilter
from ptls.data_load.datasets import MemoryMapDataset

from sklearn.model_selection import train_test_split

from utils.encode import encode_data
from utils.evaluation import bootstrap_eval

In [3]:
def evaluate_checkpoints(dataset):
    assert dataset in ["churn", "age"]

    df = pd.read_parquet(f"data/preprocessed_new/{dataset}.parquet")

    preprocessor = PandasDataPreprocessor(
        col_id="user_id",
        col_event_time="timestamp",
        event_time_transformation="dt_to_timestamp" if dataset == "churn" else "none",
        cols_category=["mcc_code"],
        cols_first_item=["global_target"]
    )

    data = preprocessor.fit_transform(df)

    val_size = 0.1
    test_size = 0.1

    train, val_test = train_test_split(data, test_size=test_size+val_size, random_state=42)
    val, test = train_test_split(val_test, test_size=test_size/(test_size+val_size), random_state=42)

    min_seq_len = 15 if dataset == "churn" else 25

    train_ds = MemoryMapDataset(train + val, [SeqLenFilter(min_seq_len=min_seq_len)])
    test_ds = MemoryMapDataset(test, [SeqLenFilter(min_seq_len=min_seq_len)])
    
    config_paths = glob.glob(f"checkpoints/{dataset}/**/*.yaml", recursive=True)
    
    results_dict = {}
    for path in config_paths:
        path_split = path.split("/")
        config_name = path_split[-1].replace(".yaml", "")
        path_to_config = "/".join(path_split[:-1])

        with initialize(version_base=None, config_path=path_to_config):
            cfg = compose(config_name=config_name)
            
        seq_encoder = hydra.utils.instantiate(cfg["seq_encoder"])
        seq_encoder.load_state_dict(torch.load(cfg["path_to_state_dict"]))

        X_train, y_train = encode_data(seq_encoder, train_ds)
        X_test, y_test = encode_data(seq_encoder, test_ds)

        results = bootstrap_eval(X_train, X_test, y_train, y_test)
        
        if not os.path.exists("results"):
            os.mkdir("results")
        
        results.to_csv(f"results/{config_name}.csv")
        
        results_dict[config_name] = results.agg(["mean", "std"]).to_dict()

    return results_dict

# Churn dataset

In [4]:
results_churn = evaluate_checkpoints("churn")

100%|██████████| 10/10 [00:38<00:00,  3.83s/it]


In [8]:
with open("results/churn.json", "w") as f:
    json.dump(results_churn, f)

# Age dataset

In [4]:
results_age = evaluate_checkpoints("age")

100%|██████████| 10/10 [06:45<00:00, 40.58s/it]


In [5]:
with open("results/age.json", "w") as f:
    json.dump(results_age, f)

# Default dataset

In [3]:
df = pd.read_parquet("data/preprocessed_new/default.parquet")

In [4]:
from utils.preprocessing import CustomDatetimeNormalization

preprocessor = PandasDataPreprocessor(
    col_id="user_id",
    col_event_time=CustomDatetimeNormalization(
        min_timestamp=1514769288,
        col_name_original="timestamp",
        col_name_target="event_time",
    ),
    cols_category=["mcc_code"],
    cols_first_item=["global_target"]
)

In [5]:
data = preprocessor.fit_transform(df)

In [6]:
y = np.array([data[i]["global_target"] for i in range(len(data))])

In [7]:
val_size = 0.1
test_size = 0.1

train, val_test, y_train, y_val_test = train_test_split(data, y, stratify=y, test_size=test_size+val_size, random_state=42)
val, test, y_val, y_test = train_test_split(val_test, y_val_test, stratify=y_val_test, test_size=test_size/(test_size+val_size), random_state=42)

In [8]:
train_ds = MemoryMapDataset(train + val, [SeqLenFilter(min_seq_len=15)])
test_ds = MemoryMapDataset(test, [SeqLenFilter(min_seq_len=15)])

In [22]:
import glob
from tqdm import tqdm

from sklearn.metrics import roc_auc_score, average_precision_score, accuracy_score
from lightgbm import LGBMClassifier


def checkpoints_eval(train_ds, test_ds, config_path, config_name):
    with hydra.initialize(version_base=None, config_path=config_path):
        cfg = hydra.compose(config_name)  

    lgbm = LGBMClassifier(
        n_estimators=500,
        boosting_type="gbdt",
        subsample=0.5,
        subsample_freq=1,
        learning_rate=0.02,
        feature_fraction=0.75,
        max_depth=6,
        lambda_l1=1,
        lambda_l2=1,
        min_data_in_leaf=50,
        random_state=42,
        n_jobs=8,
        verbose=-1
    )

    seq_encoder = hydra.utils.instantiate(cfg["seq_encoder"])

    paths = glob.glob(f'{cfg["path_to_folder"]}/*.pth')
    print(f"Found {len(paths)} checkpoints.")

    scores = []
    for path in tqdm(paths):
        seq_encoder.load_state_dict(torch.load(path))

        X_train, y_train = encode_data(seq_encoder, train_ds)
        X_test, y_test = encode_data(seq_encoder, test_ds)

        lgbm.fit(X_train, y_train)

        y_pred = lgbm.predict_proba(X_test)

        if y_pred.shape[1] == 2:
            scores.append({
                "ROC-AUC": roc_auc_score(y_test, y_pred[:, 1]),
                "PR-AUC": average_precision_score(y_test, y_pred[:, 1]),
                "Accuracy": accuracy_score(y_test, y_pred.argmax(axis=1)),
            })
        
        else:
            scores.append({
                "ROC-AUC": roc_auc_score(y_test, y_pred, average="macro", multi_class="ovr"),
                "PR-AUC": average_precision_score(y_test, y_pred, average="macro"),
                "Accuracy": accuracy_score(y_test, y_pred.argmax(axis=1))
            })


    return pd.DataFrame(scores)

In [32]:
res = checkpoints_eval(train_ds, test_ds, "checkpoints/default/ts2vec", "ts2vec_default_aligned_instance_loss")

Found 5 checkpoints.


100%|██████████| 5/5 [00:42<00:00,  8.40s/it]


In [33]:
print(res)
res.agg(["mean", "std"])

    ROC-AUC    PR-AUC  Accuracy
0  0.564460  0.080117  0.963277
1  0.591755  0.049901  0.963277
2  0.588371  0.069799  0.963277
3  0.558877  0.051763  0.963277
4  0.561132  0.042985  0.963277


,ROC-AUC,PR-AUC,Accuracy
mean,0.572919,0.058913,0.963277
std,0.015821,0.015445,0.000000


In [32]:
from sklearn.metrics import roc_auc_score, average_precision_score
from lightgbm import LGBMClassifier

res = []

for i in range(5):
    path = f"{path_to_checkpoints}{cfg['name']}_{i}.pth"
    
    seq_encoder = hydra.utils.instantiate(cfg["seq_encoder"])
    seq_encoder.load_state_dict(torch.load(path))

    X_train, y_train = encode_data(seq_encoder, train_ds, device="cuda:1")
    X_test, y_test = encode_data(seq_encoder, test_ds, device="cuda:1")

    lgbm = LGBMClassifier(
        n_estimators=500,
        boosting_type="gbdt",
        subsample=0.5,
        subsample_freq=1,
        learning_rate=0.02,
        feature_fraction=0.75,
        max_depth=6,
        lambda_l1=1,
        lambda_l2=1,
        min_data_in_leaf=50,
        random_state=42,
        n_jobs=8,
        verbose=-1
    )

    lgbm.fit(X_train, y_train)

    y_pred = lgbm.predict_proba(X_test)[:, 1]

    res.append(
        {
            "ROC-AUC": roc_auc_score(y_test, y_pred),
            "PR-AUC": average_precision_score(y_test, y_pred)
        }
    )

In [34]:
pd.DataFrame(res).agg(["mean", "std"])

,ROC-AUC,PR-AUC
mean,0.564945,0.051488
std,0.032395,0.010189
